# Non-Determinism

Refereces:
- [List monad](http://learnyouahaskell.com/a-fistful-of-monads#the-list-monad)

## Simulating chess

So let's say that the knight starts at (6,2). Can he get to (6,1) in exactly three moves? Let's see. If we start off at (6,2) what's the best move to make next? I know, how about all of them! We have non-determinism at our disposal, so instead of picking one move, let's just pick all of them at once. Here's a function that takes the knight's position and returns all of its next moves:

In [2]:
type KnightPos = (Int,Int)  

moveKnight :: KnightPos -> [KnightPos]  
moveKnight (c,r) = filter onBoard  
    [(c+2,r-1),(c+2,r+1),(c-2,r-1),(c-2,r+1)  
    ,(c+1,r-2),(c+1,r+2),(c-1,r-2),(c-1,r+2)  
    ]  
    where onBoard (c,r) = c `elem` [1..8] && r `elem` [1..8]  

In [3]:
moveKnight (6,2)

[(8,1),(8,3),(4,1),(4,3),(7,4),(5,4)]

In [4]:
moveKnight (8,1)

[(6,2),(7,3)]

In [5]:
in3 :: KnightPos -> [KnightPos]  
in3 start = do   
    first <- moveKnight start  
    second <- moveKnight first  
    moveKnight second  

This is the same as
```haskell
in3 start = return start >>= moveKnight >>= moveKnight >>= moveKnight
```

In [6]:
in3 (6,2)

[(8,1),(8,3),(4,1),(4,3),(7,4),(5,4),(5,2),(5,4),(8,1),(8,5),(6,1),(6,5),(8,1),(8,3),(4,1),(4,3),(7,4),(5,4),(8,3),(8,5),(4,3),(4,5),(7,2),(7,6),(5,2),(5,6),(5,2),(8,3),(6,3),(5,4),(5,6),(8,3),(8,7),(6,3),(6,7),(8,1),(8,3),(4,1),(4,3),(7,4),(5,4),(4,1),(4,3),(3,4),(1,4),(7,2),(7,4),(3,2),(3,4),(6,1),(6,5),(4,1),(4,5),(5,2),(5,4),(1,2),(1,4),(4,1),(4,5),(2,1),(2,5),(8,1),(8,3),(4,1),(4,3),(7,4),(5,4),(8,3),(8,5),(4,3),(4,5),(7,2),(7,6),(5,2),(5,6),(4,1),(4,3),(3,4),(1,4),(4,3),(4,5),(3,2),(3,6),(1,2),(1,6),(7,2),(3,2),(6,3),(4,3),(7,4),(7,6),(3,4),(3,6),(6,3),(6,7),(4,3),(4,7),(5,2),(1,2),(4,3),(2,3),(5,4),(5,6),(1,4),(1,6),(4,3),(4,7),(2,3),(2,7),(7,2),(7,4),(3,2),(3,4),(6,1),(6,5),(4,1),(4,5),(7,4),(7,6),(3,4),(3,6),(6,3),(6,7),(4,3),(4,7),(6,1),(6,3),(7,4),(6,5),(6,7),(7,4),(7,8),(8,1),(8,3),(4,1),(4,3),(7,4),(5,4),(8,5),(8,7),(4,5),(4,7),(7,4),(7,8),(5,4),(5,8),(5,2),(5,4),(8,1),(8,5),(6,1),(6,5),(5,4),(5,6),(8,3),(8,7),(6,3),(6,7),(5,2),(5,4),(1,2),(1,4),(4,1),(4,5),(2,1),(2,5),(5,

So to answer the question, we can check all the possible outcomes:

In [7]:
canReachIn3 :: KnightPos -> KnightPos -> Bool  
canReachIn3 start end = end `elem` in3 start  

In [8]:
canReachIn3 (6,2) (6,1)

True

In [9]:
(6,2) `canReachIn3` (6,1)

True

## Coin tossing

In [10]:
import Control.Monad

data CoinType = Fair | Biased deriving (Show)
data Coin = Head | Tail deriving (Eq,Show)

toss Fair   = [Head, Tail]
toss Biased = [Head, Head]

pick = [Fair, Biased]

experiment = do
  coin   <- pick         -- Pick a coin at random
  result <- toss coin    -- Toss it, to get a result
  guard (result == Head) -- We only care about results that come up Heads
  return coin            -- Return which coin was used in this case


In [11]:
experiment

[Fair,Biased,Biased]

## Simulating random walk
The mathematical model is a one parameter process:

$$
dx_t=\sigma dw_t
$$

where $\sigma$ is the variance, measuring the level of the noise, $dw_t$ is the white noise, with normal distribution $N(0,dt)$

The discrete form is

$$
\Delta x_i=\sigma \Delta w_i
$$

where $\Delta x_i=x_{t_i}-x_{t_{i-1}}$ and $\Delta w_i\sim N(0,\Delta t_i)$. For implementation, the following form is used, given $x_0$ and $\sigma$,

$$
x_i=x_{i-1}+\sigma\sqrt{\Delta t_i}\xi_i
$$

where $\xi_i\sim N(0,1)$

In [12]:
import Data.Random